In [1]:
from pandas import read_csv
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import shapely.wkt
from shapely.geometry import MultiPolygon
import geopandas as gpd
import contextily as cx
import csv
from libpysal import weights
import networkx as nx
import geonetworkx as gnx
import warnings
warnings.filterwarnings('ignore')
csv.field_size_limit(sys.maxsize)

os.chdir('/Users/xiaodanxu/Library/CloudStorage/GoogleDrive-arielinseu@gmail.com/My Drive/GEMS/BILD-AQ')

plt.style.use('ggplot')

In [2]:
# define scenario/ analysis boundary
# state = 'AR'
microtype_lookup = read_csv('data/ccst_geoid_key_tranps_geo_with_imputation.csv', sep = ',')
state_list = microtype_lookup['st_code'].unique()
microtype_lookup_short = microtype_lookup[['st_code', 'GEOID']]
microtype_lookup_short['GEOID'] = microtype_lookup_short['GEOID'].astype(int)
microtype_lookup_short.head(5)
# microtype_by_state = microtype_lookup.loc[microtype_lookup['st_code'] == state]
# select_tracts = microtype_by_state.GEOID.unique()
# print(len(select_tracts))

,st_code,GEOID
0,CA,6037535001
1,CA,6037535102
2,CA,6037535101
3,CA,6037535300
4,CA,6037535200


In [12]:
# process second sources of routes
all_files = os.listdir("/Volumes/LaCie/project_backup/GEMS_ROUTE/route/raw")    
gz_files = list(filter(lambda f: f.endswith('.gz'), all_files))
# print(gz_files)

# select_tracts = select_tracts.astype(int)
chunk_size = 100000
for file in gz_files:
    if file.startswith('._'):
        continue
    filename = file.split('.gz')[0]
    print(file)
#     aws_route = None
    i = 0
    for chunk in read_csv("/Volumes/LaCie/project_backup/GEMS_ROUTE/route/raw/" + file, sep = ',', chunksize = chunk_size):
        outfile_name = filename + '_' + str(i) + '.geojson'
        if i % 10 == 0:
            print('processing chunk ' + str(i))
        chunk.columns = ['source', 'destination', 'distance', 'ascend', 'descend', 'geometry']
        chunk.loc[:, ['source', 'destination']] = chunk.loc[:, ['source', 'destination']].astype(int)
        chunk = pd.merge(chunk, microtype_lookup_short,
                         left_on = 'source', right_on = 'GEOID',
                         how = 'left')
        chunk = chunk.rename(columns = {'st_code': 'orig_st'})
        chunk = chunk.drop(columns = ['GEOID'])
        
        chunk = pd.merge(chunk, microtype_lookup_short,
                         left_on = 'destination', right_on = 'GEOID',
                         how = 'left')
        chunk = chunk.rename(columns = {'st_code': 'dest_st'})
        chunk = chunk.drop(columns = ['GEOID'])
        list_of_states = chunk.orig_st.unique()
        for st in list_of_states:
            if pd.isna(st):
                continue
            chunk_selected = chunk.loc[chunk['orig_st'] == st]
            chunk_selected['geometry'] = chunk_selected['geometry'].apply(shapely.wkt.loads)
            chunk_selected = gpd.GeoDataFrame(chunk_selected, crs='epsg:4326')
            chunk_internal = chunk_selected.loc[chunk_selected['dest_st'] == st]
            chunk_external = chunk_selected.loc[chunk_selected['dest_st'] != st]
            
            internal_path = \
            '/Volumes/LaCie/project_backup/GEMS_ROUTE/route/internal/' + st + '/'
            isExist = os.path.exists(internal_path)
            if not isExist:
           # Create a new directory because it does not exist
               os.makedirs(internal_path)
            if len(chunk_internal) > 0:
                chunk_internal.to_file(internal_path + outfile_name, driver='GeoJSON')
            
            external_path = \
            '/Volumes/LaCie/project_backup/GEMS_ROUTE/route/external/' + st + '/'
            isExist = os.path.exists(external_path)
            if not isExist:
           # Create a new directory because it does not exist
               os.makedirs(external_path)
            if len(chunk_external) > 0:
                chunk_external.to_file(external_path + outfile_name, driver='GeoJSON')
      
        i += 1
#         break
#     break
# break
# chunk.head(5)

result-2-csv-2.gz
processing chunk 0
processing chunk 10
processing chunk 20
result-2-csv-3.gz
processing chunk 0
result-2-csv.gz
processing chunk 0
processing chunk 10
processing chunk 20
result-3-csv-5.gz
processing chunk 0
result-3-csv-7.gz
processing chunk 0
result-3-csv-8.gz
processing chunk 0
result-3-csv.gz
processing chunk 0
result-4-csv-2.gz
processing chunk 0
result-4-csv-3.gz
processing chunk 0
result-4-csv-4.gz
processing chunk 0
result-4-csv.gz
processing chunk 0
result-5-csv-2.gz
processing chunk 0
result-5-csv-3.gz
processing chunk 0
result-5-csv-4.gz
processing chunk 0
result-5-csv-5.gz
processing chunk 0
result-5-csv-6.gz
processing chunk 0
result-5-csv-7.gz
processing chunk 0
result-5-csv-8.gz
processing chunk 0
result-5-csv.gz
processing chunk 0
result-6-csv-0.gz
processing chunk 0
result-6-csv-2.gz
processing chunk 0
result-6-csv-3.gz
processing chunk 0
result-6-csv-4.gz
processing chunk 0
processing chunk 10
result-6-csv-5.gz
processing chunk 0
result-6-csv-6.gz
pr

In [5]:
chunk.head(5)

,source,destination,distance,ascend,descend,geometry
950000,48121021621,48085030408,24226.623038,232.009995,176.009995,"LINESTRING (-96.915238 32.998902, -96.915229 3..."
950001,48121021621,48085030504,18672.272746,191.500000,132.000000,"LINESTRING (-96.915238 32.998902, -96.915229 3..."
950002,48121021621,48085030505,20538.616387,207.000000,141.500000,"LINESTRING (-96.915238 32.998902, -96.915229 3..."
950003,48121021621,48085030506,21256.621160,215.000000,144.000000,"LINESTRING (-96.915238 32.998902, -96.915229 3..."
950004,48121021621,48085030507,21772.767564,201.000000,139.000000,"LINESTRING (-96.915238 32.998902, -96.915229 3..."


In [11]:
print(pd.isna(st))

False
